# Model Predictive Control (6) - NLP vs QP 

## Table of contents<a class="anchor" id="toc"></a> 

### [NLP vs QP](#ThisNotebook)
[Quadratic Programming Problems](#QP) / [The CVXPY QP solver](#cvxpy) /[Generating the problem](#generate) / [Solving the QP problem using cvxpy](#solvingBycvxpy) / [Solving the problem using general purpose solver IPOPT via Casadi](#solvingByCasadi)


In [1]:
from IPython.display import Image, IFrame
import ipywidgets as wg
import numpy as np
import matplotlib.pyplot as plt
import time
from casadi import MX, vertcat, Function, nlpsol, diag
import matplotlib.pyplot as plt
import cvxpy as cp


files_root = "https://www.mazenalamir.fr/MPC/"

width=800

## This Notebook<a class="anchor" id="ThisNotebook"></a>

## Quadratic Programming Problems<a class="anchor" id="QP"></a>

In the previous notebook, it has been shown how to design a General Nonlinear MPC control for any system that is given by general set of nonlinear ordinary differential equations and using a general expression of the constraints function. 

It is a fact however that in amny situation, people work with a **linearized model** around some operating point and would like to handle **constraints that are affine in the state and the control vectors**. Moreover, it is very often that the cost function is expressed via a **quandratic form of the state and the control trajectories**. 

More precisely, the system is given by (in discrete-time form):

$$
x(k+1)=Ax(k)+Bu(k)
$$

The cost function is of the form:

$$
J(\mathbf u\ \vert x(k)) := \|x(k+N)-x_d\|_{Q_f}^2+\sum_{k=0}^{N-1}\|x(k+1)-x_d\|_Q^2+\|u(k)-u_d\|_R^2
$$

While the constraints are given by:

$$
C_cx(k+i+1)+D_c u(k+i)\le \ell_c\qquad \forall i\in\{0,\dots,N-1\}
$$

It can be proved (see later) that in this case, the optimal control problem that is to be repetitively solved to implement the MPC control takes the form **This is called a quadratic programming problem (QP)**:

$$
\min_z \Bigl[\dfrac{1}{2}z^THz+h^Tz\Bigr]\quad \text{under}\quad A_{ineq}z\le B_{ineq}
$$

Now obviously this last problem can be solved using the general purpose framework CasaDi already used in the previous notebook. However:

> It is the purpose of this notebook to show that this is not a good idea since dedicated solver especially tailored to solve QPs are much more performant than general purpose NLP solver.  

The current notebook is dedicated to show this simple fact through an illustrative example. 

The QP solver that is used here to show this fact is the **cvxpy** module and the **qp/qpoases option of CasaDi**.


[back to toc](#toc)

## The CVXPY solver<a class="anchor" id="cvxpy"></a>

The following link explains how to use the cvxpy solver for a variety of problems such as:

- Linear least squares 
- Linear programs (linear cost under linear constraints
- Quadratic Programs 
- Mixed-integer Quadratic Programs
- Portfolio optimization
- and some other stuffs

In [2]:
width=700
casadi_site = "https://www.cvxpy.org/"
display(IFrame(casadi_site, width=width, height=400))

## Generating the problem<a class="anchor" id="generate"></a>

The following function 

In [3]:
# Random QP problem generator

def generate_QP(n, nc, eps):
    
    V = np.random.rand(n,n)
    Aineq = np.random.rand(nc,n)
    Bineq = 0.01+np.random.rand(nc)
    
    H = np.matmul(V.T, V)+eps*np.eye(n)
    h = 10 * V[0,:]
    
    return H, h, Aineq, Bineq
    
    
n, nc, eps = 50, 170, 0.1    
H, h, Aineq, Bineq = generate_QP(n,nc,eps)

## Solving the problem By CVXPY <a class="anchor" id="solvingBycvxpy"></a>

In [4]:
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, H) + h.T @ x),
                 [Aineq @ x <= Bineq])


Ntrials = 100
t1 = time.time()
for i in range(Ntrials):
    prob.solve()
t2 = time.time()
cpu_qp = (t2-t1)/Ntrials
print(f'value at optimum = {prob.value}')
print(f'cpu for a single computation : {cpu_qp}')
print(f'Number of active constraints = {np.sum(Aineq @ x.value - Bineq > 0)}')

value at optimum = -29.46788012282316
cpu for a single computation : 0.0035052990913391115
Number of active constraints = 16


## Solving the problem By the NLP solver Casadi/Ipopt <a class="anchor" id="solvingByCasadi"></a>

In [5]:
x_casadi = MX.sym('x', n)
J = MX.sym('J', 1)
J = 0
g = []
lbg = []
ubg = []
lbx = [-1e22] * n
ubx = [+1e22] * n
#--------------------------------------
# Define the cost function 
#--------------------------------------
linTerm = 0.0
for i in range(n):
    Hx = 0.0
    linTerm += h[i] * x_casadi[i]
    for j in range(n):
        Hx += H[i,j] * x_casadi[j]        
    J += 0.5 * Hx * x_casadi[i]
J += linTerm
#--------------------------------------
# Define the inequality constraints
#--------------------------------------
for i in range(nc):
    Aineqx = 0.0
    for j in range(n):
        Aineqx += Aineq[i,j] * x_casadi[j]
    g += [Aineqx-Bineq[i]]
    lbg += [-1e22]
    ubg += [0.0]

In [6]:
#--------------------------------------    
# Declaring the problem 
#--------------------------------------
prob = {'f':J, 'x':x_casadi, 'g':vertcat(*g)}

#--------------------------------------
# declaring the solver 
#--------------------------------------
solver = nlpsol('solver', 'ipopt', prob, {'ipopt':{'max_iter':200}})    

In [7]:
x0=np.zeros(n)


Ntrials = 10
t1 = time.time()
for i in range(Ntrials):
    sol = solver(x0=x0, lbx=lbx, ubx=ubx, lbg=lbg, ubg=ubg)
t2 = time.time()
cpu_ipopt = (t2-t1)/Ntrials


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     8500
Number of nonzeros in Lagrangian Hessian.............:     1275

Total number of variables............................:       50
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     8500
Number of nonzeros in Lagrangian Hessian.............:     1275

Total number of variables............................:       50
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:      170
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      170

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 0.00e+00 2.38e+00  -1.0 0.00e+00    -  0.00e+00 0.

  14 -2.9467880e+01 0.00e+00 2.51e-14  -8.6 1.43e-04    -  1.00e+00 1.00e+00h  1
  15 -2.9467880e+01 0.00e+00 1.87e-14  -9.0 7.85e-06    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 15

                                   (scaled)                 (unscaled)
Objective...............:  -2.9467880221974376e+01   -2.9467880221974376e+01
Dual infeasibility......:   1.8651746813702630e-14    1.8651746813702630e-14
Constraint violation....:   0.0000000000000000e+00    0.0000000000000000e+00
Complementarity.........:   9.6078661769862470e-10    9.6078661769862470e-10
Overall NLP error.......:   9.6078661769862470e-10    9.6078661769862470e-10


Number of objective function evaluations             = 16
Number of objective gradient evaluations             = 16
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 16
Number of equality constraint Jacobian evaluations   = 0
Number of inequality constraint Jacobian evaluations = 16
Num

In [8]:
optimal_value = sol['f']
x_casadi_opt = sol['x'].full().reshape(-1,1)[:,0]
constraints = np.matmul(Aineq,x_casadi_opt) - Bineq
print(f'value at optimum = {optimal_value}')
print(f'cpu for a single computation : {cpu_ipopt}')
print(f'Number of active constraints = {np.sum( constraints > 0)}')
print(f'maximum difference over components = {abs(x.value-x_casadi_opt).max()}')
print(f'Time ratio  = {cpu_ipopt/cpu_qp}')

value at optimum = -29.4679
cpu for a single computation : 0.10954492092132569
Number of active constraints = 18
maximum difference over components = 1.5234159767008038e-07
Time ratio  = 31.25123365135635


## Casadi QP (QP)  solver QPoases<a class="anchor" id="casadiQP"></a>

It is also possible to specify the fact the problem to be solved is quadratic in the CASADI framework. However, the syntax seems to me too complex and it is probably a good option to use **cvxpy** to solve QP problems. 

In [9]:
width=700
casadi_site = "https://web.casadi.org/docs/#quadratic-programming"
display(IFrame(casadi_site, width=width, height=400))

In [11]:
#%load_ext watermark
#print("----")
#%watermark -v -m -p IPython,ipywidgets,casadi,scipy,matplotlib,cvxpy
#print("----")
#%watermark -u -n -t -z